# <center> Бриф 3. Определение уязвимых групп населения
## <center> Итоговый проект первого года обучения Skillfactory Data Science

## <center> Часть 1. Формирование исходного датасета

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re
import statistics
import pickle

## 1. Анализ и обработка данных

In [2]:
# создаем словарь, к этим именам будем приводить все данные
standard_names = {
    'алтайск': 'Алтайский край',
    'амур': 'Амурская область',
    'архангел': 'Архангельская область без Ненецкого АО',
    'астрахан':  'Астраханская область',
    'белгород':  'Белгородская область',
    'брянск': 'Брянская область',
    'владимир': 'Владимирская область',
    'волгоград':  'Волгоградская область',
    'вологод': 'Вологодская область',
    'воронеж': 'Воронежская область',
    'еврей':  'Еврейская автономная область',
    'забайкал': 'Забайкальский край',
    'иванов': 'Ивановская область',
    'иркутск':  'Иркутская область',
    'кабардин': 'Кабардино-Балкарская Республика',
    'калининград': 'Калининградская область',
    'калуж': 'Калужская область',
    'камчатск(?!о)': 'Камчатский край',
    'карач': 'Карачаево-Черкесская Республика',
    'кемеров': 'Кемеровская область',
    'киров': 'Кировская область',
    'костром': 'Костромская область',
    'краснодар': 'Краснодарский край',
    'краснояр': 'Красноярский край',
    'курган': 'Курганская область',
    'курск': 'Курская область',
    'ленин': 'Ленинградская область',
    'липецк': 'Липецкая область',
    'магадан': 'Магаданская область',
    'москов': 'Московская область',
    'мурман': 'Мурманская область',
    '(?<!-)ненец': 'Ненецкий автономный округ',
    'нижегород': 'Нижегородская область',
    'новгород': 'Новгородская область',
    'новосибир': 'Новосибирская область',
    '(?<!т)омск': 'Омская область',
    'оренбург': 'Оренбургская область',
    'орлов': 'Орловская область',
    'пенз': 'Пензенская область',
    'пермск(?!о)': 'Пермский край',
    'примор': 'Приморский край',
    'псков': 'Псковская область',
    'адыг': 'Республика Адыгея',
    'алтай(?!с)': 'Республика Алтай',
    'башк': 'Республика Башкортостан',
    'бурят(?!с)': 'Республика Бурятия',
    'дагестан': 'Республика Дагестан',
    'ингуш': 'Республика Ингушетия',
    'калмык': 'Республика Калмыкия',
    'карел': 'Республика Карелия',
    'коми(?![-н])': 'Республика Коми',
    'крым(?!с)': 'Республика Крым',
    'мари': 'Республика Марий Эл',
    'морд': 'Республика Мордовия',
    'саха(?!л)': 'Республика Саха (Якутия)',
    'якут': 'Республика Саха (Якутия)',
    'осет': 'Республика Северная Осетия-Алания',
    'татар': 'Республика Татарстан',
    'тыва': 'Республика Тыва',
    'хакас': 'Республика Хакасия',
    'ростов': 'Ростовская область',
    'рязан': 'Рязанская область',
    'самар': 'Самарская область',
    'саратов': 'Саратовская область',
    'сахалин': 'Сахалинская область',
    'свердлов': 'Свердловская область',
    'смолен': 'Смоленская область',
    'ставрополь': 'Ставропольский край',
    'тамбов': 'Тамбовская область',
    'твер': 'Тверская область',
    'томск': 'Томская область',
    'туль': 'Тульская область',
    'тюмен': 'Тюменская область без округов',
    'удмурт': 'Удмуртская Республика',
    'ульянов': 'Ульяновская область',
    'хабаровск': 'Хабаровский край',
    'хант': 'Ханты-Мансийский автономный округ - Югра',
    'челябинск': 'Челябинская область',
    'чечен': 'Чеченская Республика',
    'чуваш': 'Чувашская Республика',
    'чукот': 'Чукотский автономный округ',
    'ямал': 'Ямало-Ненецкий автономный округ',
    'ярослав': 'Ярославская область',
    'москва': 'Москва',
    'петербург': 'Санкт-Петербург',
    'севастополь': 'Севастополь'}

In [3]:
# Cоздаем функцию смены названия региона
def get_standard_name(name):
    # обходим наш словарь
    for n in standard_names: 
        # имя региона приводим к нижнему регистру и
        # проверяем есть ли оно в словаре
        if n in name.lower():
            # если нашли, то вернем имя из словаря
            return standard_names[n]
    # если ничего не найдено, 
    # то возвращаем переданное 
    # в функцию имя    
    return name

In [4]:
def federal_subject(df, standard_names, names_to_drop=None, region_col='region'):
    """Функция считывает данные по федеральным субъектам

    Args:
        df (DataFrame): таблица, содержащая характеристики объектов по различным признакам
        standard_names (str): словарь со стандартным набором наименований субъектов РФ
        names_to_drop (str): список удаляемых субъектов из словаря
        region_col (str): наименование колонок с регионами
        
    Returns:
        df (DataFrame): таблица, содержащая характеристики объектов по различным признакам
    """
    def get_name(x):
        """Функция для .apply()
        Просмотр столбца регионов и вывод названий в стандартном виде"""
    
        for key, value in standard_names.items():
            x = re.sub('[\(].*?[\)]', '', x)  # удаляем всё, что в скобках ()
            if re.search(key, x.lower()):
                return value
        return None 
    if names_to_drop is None:
        names_to_drop = []
    df = df[~df[region_col].isin(names_to_drop)].copy()
    df[region_col] = df[region_col].apply(get_name)
    return (df.dropna(subset=region_col)
            .set_index(region_col).sort_index())

#### 1.1. Среднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная зарплата, по регионам

In [5]:
def finances_readout(file, sheet):
    "Считывание финансовых данных с excel-листа"
    df = pd.read_excel('data/cash_real_income_wages_2015_2020.xlsx', sheet_name=sheet)
    names_to_drop = ['Архангельская область', 'Тюменская область']
    df = federal_subject(df, standard_names, names_to_drop)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df
    
per_capita = finances_readout('data/cash_real_income_wages_2015_2020.xlsx', 0)
real_incomes = finances_readout('data/cash_real_income_wages_2015_2020.xlsx', 1)
formal_wage = finances_readout('data/cash_real_income_wages_2015_2020.xlsx', 2)
real_pay = finances_readout('data/cash_real_income_wages_2015_2020.xlsx', 3)

display('Номинальный доход на душу населения', per_capita.head(3))
display('Реальный доход, %', real_incomes.head(3))
display('Номинальная зарплата', formal_wage.head(3))
display('Реальная зарпалата, %', real_pay.head(3))

'Номинальный доход на душу населения'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20860.0,21256.0,22139.0,22829.0,23937.0,23864.0
Амурская область,28240.0,27976.0,29213.0,30937.0,33304.0,35499.0
Архангельская область без Ненецкого АО,29716.0,29837.0,30707.0,32054.0,33874.0,34852.0


'Реальный доход, %'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,99.1,94.7,100.0,99.7,99.6,95.5
Амурская область,96.1,92.1,101.1,102.4,101.7,100.3
Архангельская область без Ненецкого АО,95.1,93.0,98.7,101.7,100.2,98.4


'Номинальная зарплата'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,20090.0,21202.0,22743.0,25519.0,27962.0,30072.0
Амурская область,32902.0,33837.0,37368.0,42315.0,47234.0,52430.0
Архангельская область без Ненецкого АО,35592.0,38118.0,40352.0,45427.0,49435.0,52779.0


'Реальная зарпалата, %'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,90,98.4,103.6,109.3,104.9,103.8
Амурская область,88,96.0,107.4,110.1,106.0,105.2
Архангельская область без Ненецкого АО,92.4,100.0,102.5,110.7,104.0,102.9


In [6]:
print('Количество дубликатов признака "Номинальный доход на душу населения": {}'.format(per_capita[per_capita.duplicated()].shape[0]))
print('Количество дубликатов признака "Реальный доход": {}'.format(real_incomes[real_incomes.duplicated()].shape[0]))
print('Количество дубликатов признака "Номинальная зарплата": {}'.format(formal_wage[formal_wage.duplicated()].shape[0]))
print('Количество дубликатов признака "Реальная зарплата": {}'.format(real_pay[real_pay.duplicated()].shape[0]))

Количество дубликатов признака "Номинальный доход на душу населения": 0
Количество дубликатов признака "Реальный доход": 0
Количество дубликатов признака "Номинальная зарплата": 0
Количество дубликатов признака "Реальная зарплата": 0


In [7]:
# преобразование столбцов в формат float признаков
def type_conversion (df):
    cols = df.select_dtypes(exclude=['float']).columns
    df[cols] = df[cols].apply(pd.to_numeric, downcast='float', errors='coerce')
    return df[cols]
real_pay = type_conversion(real_pay)

In [8]:
# усредним данные признака по годам
per_capita_mean = per_capita.mean(axis=1).round(1)
real_incomes_mean = real_incomes.mean(axis=1).round(1)
formal_wage_mean = formal_wage.mean(axis=1).round(1)
real_pay_mean = real_pay.mean(axis=1).round(1)

In [9]:
display(per_capita_mean, real_incomes_mean, formal_wage_mean, real_pay_mean)

region
Алтайский край                            22480.8
Амурская область                          30861.5
Архангельская область без Ненецкого АО    31840.0
Астраханская область                      23899.5
Белгородская область                      30692.5
                                           ...   
Чеченская Республика                      23330.5
Чувашская Республика                      18919.7
Чукотский автономный округ                75961.8
Ямало-Ненецкий автономный округ           78070.5
Ярославская область                       27634.7
Length: 85, dtype: float64

region
Алтайский край                             98.1
Амурская область                           98.9
Архангельская область без Ненецкого АО     97.9
Астраханская область                       96.6
Белгородская область                       99.4
                                          ...  
Чеченская Республика                       99.1
Чувашская Республика                       98.9
Чукотский автономный округ                101.5
Ямало-Ненецкий автономный округ           102.2
Ярославская область                        97.4
Length: 85, dtype: float64

region
Алтайский край                            24598.0
Амурская область                          41014.3
Архангельская область без Ненецкого АО    43617.2
Астраханская область                      31866.5
Белгородская область                      30920.3
                                           ...   
Чеченская Республика                      25364.0
Чувашская Республика                      26226.3
Чукотский автономный округ                97464.2
Ямало-Ненецкий автономный округ           93313.3
Ярославская область                       32216.3
Length: 85, dtype: float64

region
Алтайский край                            90.000000
Амурская область                          88.000000
Архангельская область без Ненецкого АО    92.400002
Астраханская область                      89.900002
Белгородская область                      93.199997
                                            ...    
Чеченская Республика                      88.199997
Чувашская Республика                      89.599998
Чукотский автономный округ                96.300003
Ямало-Ненецкий автономный округ           94.099998
Ярославская область                       89.800003
Length: 85, dtype: float32

#### 1.2. Детская смертность

In [10]:
def infant_mortality(file):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, header=2, skipfooter=13, usecols='A, D:AI')
    df.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
    names_to_drop = [
        '            Архангельская область (кроме Ненецкого автономного округа)',
        '            Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    ]
    df = federal_subject(df, standard_names, names_to_drop)
    df.loc['Архангельская область без Ненецкого АО'] \
        = (df.loc['Архангельская область без Ненецкого АО']
           - df.loc['Ненецкий автономный округ'])
    df.loc['Тюменская область без округов'] \
        = (df.loc['Тюменская область без округов']
           - df.loc['Ханты-Мансийский автономный округ - Югра']
           - df.loc['Ямало-Ненецкий автономный округ'])
    df.columns = pd.to_datetime(df.columns, format='%Y').year
    return df

child_mortality_rural = infant_mortality('data/child_mortality_rural_1990_2021.xls')
child_mortality_rural.head(3)

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,217.0,245.0,230.0,249.0,226.0,261.0,220.0,213.0,176.0,196.0,...,173.0,154.0,117.0,94.0,87.0,79.0,70.0,60.0,44.0,42.0
Амурская область,142.0,124.0,106.0,108.0,111.0,99.0,100.0,118.0,86.0,91.0,...,63.0,50.0,39.0,31.0,23.0,20.0,17.0,16.0,13.0,15.0
Архангельская область без Ненецкого АО,105.0,108.0,82.0,91.0,74.0,56.0,60.0,42.0,49.0,51.0,...,31.0,29.0,29.0,25.0,17.0,22.0,14.0,16.0,5.0,10.0


In [11]:
child_mortality_urban = infant_mortality('data/child_mortality_urban_1990_2021.xls')
child_mortality_urban.head(3)

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,305.0,322.0,299.0,311.0,271.0,229.0,185.0,162.0,164.0,195.0,...,157.0,136.0,158.0,128.0,124.0,106.0,106.0,54.0,53.0,46.0
Амурская область,201.0,169.0,174.0,136.0,131.0,151.0,145.0,150.0,160.0,179.0,...,98.0,69.0,62.0,52.0,30.0,28.0,29.0,27.0,29.0,21.0
Архангельская область без Ненецкого АО,183.0,166.0,163.0,146.0,150.0,150.0,134.0,115.0,118.0,117.0,...,77.0,84.0,68.0,59.0,62.0,50.0,39.0,42.0,24.0,22.0


In [12]:
child_mortality = child_mortality_rural.add(child_mortality_urban)
child_mortality.head(3)

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,522.0,567.0,529.0,560.0,497.0,490.0,405.0,375.0,340.0,391.0,...,330.0,290.0,275.0,222.0,211.0,185.0,176.0,114.0,97.0,88.0
Амурская область,343.0,293.0,280.0,244.0,242.0,250.0,245.0,268.0,246.0,270.0,...,161.0,119.0,101.0,83.0,53.0,48.0,46.0,43.0,42.0,36.0
Архангельская область без Ненецкого АО,288.0,274.0,245.0,237.0,224.0,206.0,194.0,157.0,167.0,168.0,...,108.0,113.0,97.0,84.0,79.0,72.0,53.0,58.0,29.0,32.0


In [13]:
print('Количество дубликатов признака "Детская смертность": {}'.format(child_mortality[child_mortality.duplicated()].shape[0]))

Количество дубликатов признака "Детская смертность": 0


In [14]:
# усредним данные признака по годам
child_mortality_mean = child_mortality.mean(axis=1).round(1)
child_mortality_mean

region
Алтайский край                            323.7
Амурская область                          174.8
Архангельская область без Ненецкого АО    143.4
Астраханская область                      145.7
Белгородская область                      139.2
                                          ...  
Чеченская Республика                      369.0
Чувашская Республика                      135.1
Чукотский автономный округ                 19.5
Ямало-Ненецкий автономный округ            90.5
Ярославская область                       116.1
Length: 85, dtype: float64

#### 1.3. Инвалидность среди трудоспособного возраста

In [15]:
disabled = pd.read_csv('data/disabled_total_by_age_2017_2022.csv')
disabled = federal_subject(disabled, standard_names)

disabled['date'] = pd.to_datetime(disabled['date'])
disabled['dayofyear'] = disabled['date'].dt.dayofyear
disabled = disabled[disabled['dayofyear'] == 1]
disabled['young'] = disabled[['18_30', '31_40', '41_50', '51_60']].sum(axis=1)
disabled_18_60 = pd.pivot_table(data=disabled, values='young', index='region', columns='date')
disabled_18_60.columns = (disabled_18_60.columns).year
disabled_18_60.head(3)

date,2017,2018,2019,2020,2021,2022
region,,,,,,
Алтайский край,66770.0,64605.0,62840.0,61378.0,59810.0,59006.0
Амурская область,27892.0,26720.0,25976.0,25069.0,24149.0,23356.0
Архангельская область без Ненецкого АО,29228.0,28262.0,27615.0,26984.0,26498.0,26009.0


In [16]:
# усредним данные признака по годам
disabled_mean = disabled_18_60.mean(axis=1).round(1)
disabled_mean

region
Алтайский край                             62401.5
Амурская область                           25527.0
Архангельская область без Ненецкого АО     27432.7
Астраханская область                       19912.0
Белгородская область                       62646.7
                                            ...   
Чеченская Республика                      135625.3
Чувашская Республика                       29728.8
Чукотский автономный округ                  1062.8
Ямало-Ненецкий автономный округ             7700.7
Ярославская область                        29749.5
Length: 85, dtype: float64

#### 1.4. Алкоголизм, наркомания

In [17]:
def alco_readout(file, sheet):
    "Считывание данных с excel-листа"
    df = pd.read_excel(file, sheet_name=sheet)
    df = federal_subject(df, standard_names)
    df.columns = pd.to_datetime((df.columns).astype(int), format='%Y').year
    return df

alco = alco_readout('data/drug_alco.xlsx', 0)
alco1718 = alco_readout('data/drug_alco.xlsx', 1)
drug = alco_readout('data/drug_alco.xlsx', 2)
drug1718 = alco_readout('data/drug_alco.xlsx', 3)
alco = pd.concat([alco, alco1718], axis=1).sort_index()
drug = pd.concat([drug, drug1718], axis=1).sort_index()

display('Алкоголизм', alco.head(3))
display('Наркомания', drug.head(3))

'Алкоголизм'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,227.6,202.4,173.9,199.8,177.9,164.0,157.6,138.9,105.0,111.9,115.7,100.1,85.9,88.1
Амурская область,174.3,159.0,120.9,113.3,115.5,140.5,124.7,104.8,100.0,96.7,97.2,92.3,91.4,113.3
Архангельская область без Ненецкого АО,120.1,115.2,102.5,89.8,96.5,99.5,84.4,73.2,77.6,NaN,NaN,NaN,55.3,64.7


'Наркомания'

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
region,,,,,,,,,,,,,,
Алтайский край,29.8,35.2,35.2,37.7,27.3,24.5,26.4,33.4,22.9,21.6,20.7,17.4,17.4,17.4
Амурская область,21.3,27.2,27.2,28.4,26.2,37.1,33.9,24.2,22.5,22.2,19.1,18.9,18.7,24.4
Архангельская область без Ненецкого АО,1.5,1.4,1.4,2.6,3.4,4.6,6.4,4.0,4.0,NaN,NaN,NaN,5.4,9.2


In [18]:
alco = type_conversion(alco)
drug = type_conversion(drug)

# усредним данные признака по годам

alco_mean = alco.mean(axis=1)
drug_mean = drug.mean(axis=1)
print('Среднее количество впервые зарегистрированных алкогольных психозов', alco_mean.round(2))
print('Среднее количество впервые зарегистрированных наркотических психозов', drug_mean.round(2))

Среднее количество впервые зарегистрированных алкогольных психозов region
Алтайский край                             87.000000
Амурская область                          102.349998
Архангельская область без Ненецкого АО     60.000000
Астраханская область                       24.200001
Белгородская область                       43.250000
                                             ...    
Чеченская Республика                        3.550000
Чувашская Республика                       77.699997
Чукотский автономный округ                231.399994
Ямало-Ненецкий автономный округ            93.300003
Ярославская область                        56.299999
Length: 85, dtype: float32
Среднее количество впервые зарегистрированных наркотических психозов region
Алтайский край                            17.400000
Амурская область                          21.549999
Архангельская область без Ненецкого АО     7.300000
Астраханская область                       0.450000
Белгородская область            

#### 1.5. Валовый региональный продукт на душу населения

In [19]:
gross_regional_product = pd.read_excel('data/gross_regional_product_1996_2020.xls', header=2, skipfooter=1, usecols='A, Q:Z')
gross_regional_product.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область']
gross_regional_product = federal_subject(gross_regional_product, standard_names, names_to_drop)
gross_regional_product.columns = pd.to_datetime(gross_regional_product.columns).year
gross_regional_product.head(3)

,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,
Алтайский край,137677.2,153556.7,173763.5,186798.6,204933.1,224525.8,231268.4,247599.3,271319.7,291156.9
Амурская область,273175.8,280023.9,258817.0,286282.6,343385.7,370192.4,373935.1,419905.2,521060.1,571362.1
Архангельская область без Ненецкого АО,232540.7,270662.9,283264.5,310817.4,352837.9,400764.6,441961.6,493205.1,509917.0,514200.4


In [20]:
# усредним данные признака по годам
gross_regional_product_mean = gross_regional_product.mean(axis=1).round(1)
gross_regional_product_mean

region
Алтайский край                             212259.9
Амурская область                           369814.0
Архангельская область без Ненецкого АО     381017.2
Астраханская область                       374377.4
Белгородская область                       479672.1
                                            ...    
Чеченская Республика                       124347.6
Чувашская Республика                       219475.7
Чукотский автономный округ                1392794.6
Ямало-Ненецкий автономный округ           3773326.6
Ярославская область                        367890.3
Length: 85, dtype: float64

#### 1.6. Жилищные условия по данным 2020 г.

In [21]:
def housing_readout(file, sheet, cols, skipfooter=0):
    "Считывание данных с excel-листа"
    df = pd.read_excel(
        file, sheet_name=sheet, usecols=cols, skipfooter=skipfooter)
    df.rename({'Регион': 'region'}, axis=1, inplace=True)
    
    # Ошибки в написании названий 
    df.replace({'Bладимирская область': 'Владимирская'}, inplace=True)
    df.replace({'Bолгоградская область': 'Волгоградская'}, inplace=True)
    df.replace({'Bологодская область': 'Вологодская'}, inplace=True)
    df.replace({'Bоронежская область': 'Воронежская'}, inplace=True)
    df.replace('...', 0, inplace=True)  # слабая статистика -> 0
    df = federal_subject(df, standard_names)
    return df

living_area = housing_readout('data/housing_2020.xlsx', 0, 'A, H')
living_area.columns.values[0] = 'living area'
intents = housing_readout('data/housing_2020.xlsx', 1, 'A, C, E, H, I', 2)
intents.columns.values[0] = 'to improve'
intents.columns.values[1] = 'bad conditions'
intents.columns.values[2] = 'queue'
intents.columns.values[3] = 'demolition'
intents['no funds'] = ((intents['to improve']/100 
    * (intents['queue']/100 + intents['demolition']/100)) * 100)
housing = pd.concat(
    [living_area, intents[['bad conditions', 'no funds']]], axis=1)
housing.head()

C:\Users\Администратор\AppData\Local\Temp\ipykernel_13140\969773691.py:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('...', 0, inplace=True)  # слабая статистика -> 0


,living area,bad conditions,no funds
region,,,
Алтайский край,18.0,0.3,0.8642
Амурская область,17.3,1.3,1.0731
Архангельская область без Ненецкого АО,16.9,1.4,1.2998
Астраханская область,19.2,0.8,2.9028
Белгородская область,17.8,0.7,0.3840


#### 1.7. Заболеваемость

In [22]:
morbidity = pd.read_excel('data/morbidity_2005_2020_age_disease.xls', header=2, usecols='A:C, M:O')
morbidity.rename(
    {'Unnamed: 0': 'region', 'Unnamed: 1': 'type', 'Unnamed: 2': 'age'}, 
    axis=1, inplace=True)
morbidity = morbidity.dropna(subset=['2014', '2015', '2016'], how='all')
morbidity = federal_subject(morbidity, standard_names)

# Общая заболеваемость
mask = (
    (morbidity['type'] == 'Все заболевания')
    & (morbidity['age'] ==  'Всего')   
)
general_morbidity = morbidity[mask].copy()
general_morbidity.drop(columns=['type', 'age', '2014'], inplace=True)
general_morbidity.columns = pd.to_datetime(
    general_morbidity.columns).year

# Врожденные пороки развития
mask = (
    (morbidity['type'] == 'Врожденные аномалии (пороки развития), деформации и хромосомные нарушения')
    & (morbidity['age'] ==  'Всего')   
)
congenital_malformation = morbidity[mask].copy()
congenital_malformation.drop(columns=['type', 'age'], inplace=True)
congenital_malformation.columns = pd.to_datetime(
    congenital_malformation.columns).year

# Психические расстройства и расстройства поведения
mask = (
    (morbidity['type'] == 'Психические расстройства и расстройства поведения')
    & (morbidity['age'] ==  'Всего')   
)
psychosis = morbidity[mask].copy()
psychosis.drop(columns=['type', 'age'], inplace=True)
psychosis.columns = pd.to_datetime(
    psychosis.columns).year

# Подростковая беременность (до 14 лет)
mask = (
    (morbidity['type'] == 'Беременность, роды и послеродовой период')
    & (morbidity['age'] ==  '0-14 лет')   
)
teenage_pregnancy = morbidity[mask].copy()
teenage_pregnancy.drop(columns=['type', 'age'], inplace=True)
teenage_pregnancy.columns = pd.to_datetime(
    teenage_pregnancy.columns).year

display('Общая заболеваемость', general_morbidity.head(3))
display('Врожденные пороки развития', congenital_malformation.head(3))
display('Психические расстройства', psychosis.head(3))
display('Подростковая беременность', teenage_pregnancy.head(3))

'Общая заболеваемость'

,2015,2016
region,,
Алтайский край,109652.6,112764.7
Амурская область,83919.0,83412.0
Архангельская область без Ненецкого АО,101587.4,99842.1


'Врожденные пороки развития'

,2014,2015,2016
region,,,
Алтайский край,176.1,196.7,200.1
Амурская область,391.2,335.2,370.1
Архангельская область без Ненецкого АО,380.8,423.9,458.9


'Психические расстройства'

,2014,2015,2016
region,,,
Алтайский край,1454.0,1467.7,1272.1
Амурская область,512.4,485.3,446.7
Архангельская область без Ненецкого АО,646.0,693.5,685.3


'Подростковая беременность'

,2014,2015,2016
region,,,
Алтайский край,3.4,8.3,3.3
Амурская область,22.4,13.3,4.4
Архангельская область без Ненецкого АО,10.5,6.9,13.8


In [23]:
# усредним данные признака по годам
general_morbidity_mean = general_morbidity.mean(axis=1).round(1)
congenital_malformation_mean = congenital_malformation.mean(axis=1).round(1)
psychosis_mean = psychosis.mean(axis=1).round(1)
teenage_pregnancy_mean = teenage_pregnancy.mean(axis=1).round(1)

#### 1.8. Рождаемость

In [24]:
newborn = pd.read_csv('data/newborn_2006_2022_monthly.csv', sep=';', decimal=',')
newborn.drop(columns='Unnamed: 198',inplace=True)
newborn.rename({'Region': 'region'}, axis=1, inplace=True)
names_to_drop = [
    'Архангельская область (кроме Ненецкого автономного округа)',
    'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
    'Московская обл. в старых границах',
    'Москва в старых границах'
]
newborn = federal_subject(newborn, standard_names, names_to_drop=names_to_drop)
newborn.loc['Архангельская область без Ненецкого АО'] \
    = (newborn.loc['Архангельская область без Ненецкого АО']
      - newborn.loc['Ненецкий автономный округ'])
newborn.loc['Тюменская область без округов'] \
    = (newborn.loc['Тюменская область без округов']
       - newborn.loc['Ханты-Мансийский автономный округ - Югра']
       - newborn.loc['Ямало-Ненецкий автономный округ'])
    
# Суммируем данные за год
for year in range(2006, 2023):
    cols = []
    for col in newborn.columns:
        if str(year) in str(col):
            cols.append(col)
    newborn[year] = newborn[cols].sum(axis=1)
    newborn.drop(columns=cols, inplace=True)
newborn.drop(columns=2022, inplace=True)  # Неполный год
newborn = newborn.round(0).astype(int)
newborn.columns = pd.to_datetime(newborn.columns, format='%Y').year

newborn.head(3)

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,,,,,,,,
Алтайский край,25513,28209,30711,31038,30863,30739,32695,32461,31429,30065,28867,25525,23491,21117,19976,18992
Амурская область,10405,10935,11233,11384,11473,11188,11733,11483,11159,10781,10377,9430,8889,7919,7853,7343
Архангельская область без Ненецкого АО,13396,14438,14577,14776,14806,14253,14730,14584,14331,13908,13313,11714,10671,9662,8906,8487


In [25]:
# усредним данные признака по годам
newborn_mean = newborn.mean(axis=1).round(1)
newborn_mean

region
Алтайский край                            27605.7
Амурская область                          10224.1
Архангельская область без Ненецкого АО    12909.5
Астраханская область                      13403.9
Белгородская область                      15822.9
                                           ...   
Чеченская Республика                      32349.1
Чувашская Республика                      14882.6
Чукотский автономный округ                  669.4
Ямало-Ненецкий автономный округ            7957.2
Ярославская область                       13513.1
Length: 85, dtype: float64

#### 1.9. Население

In [26]:
population = pd.read_excel('data/population.xlsx', header=1, usecols='A, C:Z')
population.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)

for i, val in enumerate(population['region']):
    if val == 'Раздел 1. Муниципальные образования субъектов Российской Федерации':
        population.loc[i, 'region'] = population.loc[i-1, 'region']
population.dropna(subset=population.columns[1:], how='all', inplace=True)

names_to_drop = [
        'Архангельская область (кроме Ненецкого автономного округа)',
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)',
        'Муниципальные образования Алтайского края',
        'Алтайский муниципальный район',
        'Алтайский сельсовет',
        'Большекалтайский сельсовет',
        'Населенные пункты, входящие в состав муниципальных образований Алтайского края',
        'Кировский сельсовет'
]
population = federal_subject(population, standard_names, names_to_drop)
population.loc['Архангельская область без Ненецкого АО'] \
    = (population.loc['Архангельская область без Ненецкого АО']
      - population.loc['Ненецкий автономный округ'])
population.loc['Тюменская область без округов'] \
    = (population.loc['Тюменская область без округов']
       - population.loc['Ханты-Мансийский автономный округ - Югра']
       - population.loc['Ямало-Ненецкий автономный округ'])

newcols = []
for col in population.columns:
    newcols.append(re.search(r'\d+', col).group(0))
population.columns = pd.to_datetime(newcols).year

population.head(3)

,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,2662738.0,2651628.0,2641079.0,2621050.0,2602595.0,2571987.0,2539430.0,2503510.0,2473024.0,2453455.0,...,2398751.0,2390638.0,2384812.0,2376774.0,2365680.0,2350080.0,2332813.0,2317153.0,2296353.0,2268179.0
Амурская область,949526.0,935607.0,923055.0,911381.0,901044.0,887781.0,874018.0,861056.0,850502.0,844290.0,...,816910.0,811274.0,809873.0,805689.0,801752.0,798424.0,793194.0,790044.0,781846.0,772525.0
Архангельская область без Ненецкого АО,1372631.0,1349160.0,1328187.0,1309552.0,1290956.0,1273668.0,1257312.0,1239924.0,1224813.0,1215264.0,...,1159506.0,1148760.0,1139950.0,1130240.0,1121813.0,1111031.0,1100290.0,1092424.0,1082662.0,1069782.0


In [27]:
# усредним данные признака по годам
population_mean = population.mean(axis=1).round(1)
population_mean

region
Алтайский край                            2453999.7
Амурская область                           843491.6
Архангельская область без Ненецкого АО    1198813.4
Астраханская область                      1009587.7
Белгородская область                      1528216.2
                                            ...    
Чеченская Республика                      1271048.8
Чувашская Республика                      1263311.6
Чукотский автономный округ                  52577.1
Ямало-Ненецкий автономный округ            525722.2
Ярославская область                       1302403.5
Length: 85, dtype: float64

#### 1.10. Процент населения за чертой бедности (доходы ниже прожиточного минимума)

In [28]:
poverty_percent = pd.read_csv('data/poverty_percent_by_regions_1992_2020.csv')
poverty_percent = pd.pivot_table(
    data=poverty_percent, values='poverty_percent', 
    columns='year', index='region').reset_index()
names_to_drop = [
    '        Архангельская область',
    '        Тюменская область'
]
poverty_percent = federal_subject(poverty_percent, standard_names, names_to_drop)
poverty_percent.columns = pd.to_datetime(
    poverty_percent.columns, format='%Y').year
poverty_percent.fillna(poverty_percent.mean(), inplace=True)
poverty_percent = poverty_percent.iloc[:, 3:]
poverty_percent.head(3)

year,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,33.70000,46.80000,45.70000,52.90000,53.80000,53.90000,47.3000,38.90000,33.900000,30.900000,...,22.600000,20.6,17.6,17.1,18.0,17.8,17.5,17.4,17.6,17.5
Амурская область,36.10000,28.20000,26.30000,31.20000,38.00000,47.70000,45.3000,44.60000,35.600000,33.800000,...,20.400000,16.0,16.2,14.8,15.2,17.0,16.7,15.6,15.7,15.2
Архангельская область без Ненецкого АО,30.17875,28.47375,25.35875,29.95875,37.75625,41.16625,37.0725,32.73125,28.104938,24.846914,...,15.106173,13.0,14.5,14.5,16.2,14.9,13.9,12.5,12.7,12.3


In [29]:
# усредним данные признака по годам
poverty_percent_mean = poverty_percent.mean(axis=1).round(1)
poverty_percent_mean

region
Алтайский край                            29.0
Амурская область                          26.2
Архангельская область без Ненецкого АО    21.6
Астраханская область                      20.4
Белгородская область                      14.8
                                          ... 
Чеченская Республика                      16.6
Чувашская Республика                      26.2
Чукотский автономный округ                11.9
Ямало-Ненецкий автономный округ            5.8
Ярославская область                       16.2
Length: 85, dtype: float64

#### 1.11. Распределение малоимущего населения по социально-демографическим группам

In [30]:
def socdem_readout(file):
    df = pd.read_excel(file, header=2)
    df.rename({
        'Unnamed: 0': 'region',
        'Все население': 'all',
        'Дети в возрасте до 16 лет': 'children',
        'Население старше трудоспособного возраста': 'retired',
        'Население трудоспособного возраста': 'employable'
        }, axis=1, inplace=True)
    
    names_to_drop = [
        '        Архангельская область',
        '        Тюменская область'
    ]
    df = federal_subject(df, standard_names, names_to_drop)
    return df


poverty_socdem_2017 = socdem_readout('data/poverty_socdem_2017.xls')
poverty_socdem_2018 = socdem_readout('data/poverty_socdem_2018.xls')
poverty_socdem_2019 = socdem_readout('data/poverty_socdem_2019.xls')
poverty_socdem_2020 = socdem_readout('data/poverty_socdem_2020.xls')

poverty_socdem = poverty_socdem_2020.copy()

mask = (poverty_socdem[['children', 'retired', 'employable']].isna().all(axis=1))
poverty_socdem[mask] = poverty_socdem_2019

mask = ((poverty_socdem[['children', 'retired', 'employable']] == 0).any(axis=1))
poverty_socdem[mask] = poverty_socdem_2018

mask = ((poverty_socdem[['children', 'retired', 'employable']] == 0).any(axis=1))
poverty_socdem[mask] = poverty_socdem_2017
poverty_socdem.drop(columns=['all'], inplace=True)
poverty_socdem.head(3)

,children,retired,employable
region,,,
Алтайский край,31.4,9.2,59.4
Амурская область,38.4,4.7,56.9
Архангельская область без Ненецкого АО,28.5,7.2,64.3


#### 1.12. Региональное производство

In [31]:
regional_production = pd.read_csv('data/regional_production_2017_2020.csv')
names_to_drop = [
    '        Архангельская область', 
    '        Тюменская область']
regional_production = federal_subject(regional_production, standard_names, names_to_drop) 
regional_production.columns.values[-4:] = pd.to_datetime(regional_production.columns[-4:]).year
reg_prod_total = (regional_production
                  .groupby('region')[[2017, 2018, 2019, 2020]].sum()) 
reg_prod_total.columns = pd.to_datetime(
    reg_prod_total.columns, format='%Y').year

display('Региональное производство, по видам деятельности', regional_production.head(5))
display('Региональное производство, общий объем', reg_prod_total.head(3))

'Региональное производство, по видам деятельности'

,production_field,2017,2018,2019,2020
region,,,,,
Алтайский край,"ВОДОСНАБЖЕНИЕ; ВОДООТВЕДЕНИЕ, ОРГАНИЗАЦИЯ СБОР...",8068621.4,1.032303e+07,9813456.6,1.045441e+07
Алтайский край,ДОБЫЧА ПОЛЕЗНЫХ ИСКОПАЕМЫХ,6143029.4,5.023392e+06,7745963.3,9.146146e+06
Алтайский край,ОБРАБАТЫВАЮЩИЕ ПРОИЗВОДСТВА,307969929.4,3.228103e+08,357699013.3,3.618801e+08
Алтайский край,"ОБЕСПЕЧЕНИЕ ЭЛЕКТРИЧЕСКОЙ ЭНЕРГИЕЙ, ГАЗОМ И ПА...",45806615.7,4.743969e+07,43900541.6,4.474102e+07
Алтайский край,Промышленное производство (промышленность),367988195.7,3.855964e+08,419158974.7,4.262217e+08


'Региональное производство, общий объем'

,2017,2018,2019,2020
region,,,,
Алтайский край,735976391.6,7.711927e+08,838317949.5,8.524434e+08
Амурская область,268844147.6,2.687608e+08,340026685.8,4.045765e+08
Архангельская область без Ненецкого АО,709009752.2,6.064711e+08,533584941.4,6.641070e+08


In [32]:
# усредним данные признака по годам
reg_prod_total_mean = reg_prod_total.mean(axis=1).round(1)
reg_prod_total_mean

region
Алтайский край                            7.994826e+08
Амурская область                          3.205520e+08
Архангельская область без Ненецкого АО    6.282932e+08
Астраханская область                      7.841122e+08
Белгородская область                      1.795972e+09
                                              ...     
Чеченская Республика                      7.359043e+07
Чувашская Республика                      4.753075e+08
Чукотский автономный округ                1.874916e+08
Ямало-Ненецкий автономный округ           5.569600e+09
Ярославская область                       8.851531e+08
Length: 85, dtype: float64

#### 1.13. Розничная торговля на душу населения

In [33]:
retail = pd.read_excel('data/retail_turnover_per_capita_2000_2021.xls', header=2, usecols='A, P:X')
retail.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
retail.drop(0, inplace=True)
names_to_drop = [
        '        Архангельская область',
        '            Архангельская область (без АО)',
        '        Тюменская область',
        '            Тюменская область (без АО)'
        ]
retail = federal_subject(retail, standard_names, names_to_drop)
retail.columns = pd.to_datetime(retail.columns).year

retail.head(3)

,2013,2014,2015,2016,2017,2018,2019,2020,2021
region,,,,,,,,,
Алтайский край,118096,128376,134925,137844,143873,150444,159514,153605,174321
Амурская область,145301,163781,182491,191523,202038,214688,231113,245233,276635
Архангельская область без Ненецкого АО,154177,176420,194345,202977,217332,229922,240155,250033,280050


In [34]:
retail = type_conversion(retail)
# усредним данные признака по годам
retail_mean = retail.mean(axis=1).round()
retail_mean

region
Алтайский край                            144555.0
Амурская область                          205867.0
Архангельская область без Ненецкого АО    216157.0
Астраханская область                      169976.0
Белгородская область                      202996.0
                                            ...   
Чеченская Республика                      110248.0
Чувашская Республика                      123133.0
Чукотский автономный округ                168238.0
Ямало-Ненецкий автономный округ           249421.0
Ярославская область                       179947.0
Length: 85, dtype: float32

#### 1.14. Расходы на социальную политику

In [35]:
welfare_expense_share = pd.read_excel('data/welfare_expense_share_2015_2020.xlsx', decimal=',')
welfare_expense_share.rename({'Unnamed: 0': 'region'}, axis=1, inplace=True)
welfare_expense_share = federal_subject(welfare_expense_share, standard_names)

display('Расходы на социальную политику, млн руб.', welfare_expense_share.head())

'Расходы на социальную политику, млн руб.'

,2015,2016,2017,2018,2019,2020
region,,,,,,
Алтайский край,18.8,20.4,30.9,29.8,28.3,29.7
Амурская область,19.2,28.9,26.3,24.1,22.7,21.7
Архангельская область без Ненецкого АО,15.4,16.3,24.8,24.2,22.4,21.4
Астраханская область,18.8,20.5,27.7,26.4,25.8,31.4
Белгородская область,11.3,11.8,16.4,15.6,13.8,16.1


In [36]:
# усредним данные признака по годам
welfare_mean = welfare_expense_share.mean(axis=1).round(1)
welfare_mean

region
Алтайский край                            26.3
Амурская область                          23.8
Архангельская область без Ненецкого АО    20.8
Астраханская область                      25.1
Белгородская область                      14.2
                                          ... 
Чеченская Республика                      27.3
Чувашская Республика                      22.5
Чукотский автономный округ                 6.6
Ямало-Ненецкий автономный округ           13.1
Ярославская область                       20.9
Length: 85, dtype: float64

## Дополнительное задание
Обработка исходых данных по преступлениям 2016-2021 г.г.

#### 1.15. Преступность

In [37]:
# Cоздадим функцию обработки датасета из папки crimes
def criminal_processing(crimes):
    # Обработаем наименование и оставим только часть колонок по преступлениям
    #crimes = crimes.rename(columns={'Unnamed: 1': 'region'})
    crimes.rename({
        'Unnamed: 1': 'region',
        'Unnamed: 2': 'by minors',
        'Unnamed: 3': 'previously',
        'ранее судимыми': 'previous convictions',
        'Unnamed: 5': 'group of people',
        'Unnamed: 6': 'group by prior agreement',
        'Unnamed: 7': 'organized group',
        'Unnamed: 8': 'criminal organization',
        'алкогольного\n': 'alcohol intoxication',
        'наркотического\n': 'drug intoxication',
        'токсического\n': 'toxic intoxication'
        }, axis=1, inplace=True)
    
    crimes.drop([0, 1], inplace=True)
    
    # удаляем лишние столбцы
    crimes = crimes.drop(columns=['Unnamed: 0'])
    # удалим пробелы в названиях регионов
    crimes['region'] = crimes['region'].str.strip()
    # убираем строки содержащие "Российская Федерация", федеральный округ и район
    region_for_delete = crimes[crimes['region'].str.contains('федеральный округ|Федерация|район|УТ МВД по |Транспорт России')].index
    crimes.drop(region_for_delete, axis = 0, inplace=True)

    # меняем название регионов в соответствии с нашим словарем
    crimes['region'] = crimes['region'].apply(get_standard_name)
    
    crimes['previously committed crimes'] = crimes['previously'] + crimes['previous convictions']
    crimes['committed by the group'] = crimes['group of people'] + crimes['group by prior agreement'] + \
        crimes['organized group'] + crimes['criminal organization']
    crimes['intoxicating'] = crimes['alcohol intoxication'] + crimes['drug intoxication'] + \
        crimes['toxic intoxication']
    
    # удаляем лишние столбцы
    crimes = crimes.drop(columns=['previously', 'previous convictions', 'group of people', \
                                  'group by prior agreement', 'organized group', 'criminal organization', \
                                  'alcohol intoxication', 'drug intoxication', 'toxic intoxication'])
    crimes.set_index('region', inplace=True)
    return crimes

In [38]:
crimes_2020 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122020.xls', skiprows=10, sheet_name='Строка 1') 
crimes_2019 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122019.xls', skiprows=10, sheet_name='Строка 1') 
crimes_2018 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122018.xls', skiprows=10, sheet_name='Строка 1')
crimes_2017 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122017.xls', skiprows=10, sheet_name='Строка 1')
crimes_2016 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122016.xls', skiprows=10, sheet_name='Строка 1')

# обработаем все датасеты из папки crimes
crimes_2020 = criminal_processing(crimes_2020)
crimes_2019 = criminal_processing(crimes_2019)
crimes_2018 = criminal_processing(crimes_2018)
crimes_2017 = criminal_processing(crimes_2017)
crimes_2016 = criminal_processing(crimes_2016)


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (122053) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (122053) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [39]:
crimes_2016['total_crimes'] = crimes_2016.sum(axis = 1)
crimes_2017['total_crimes'] = crimes_2017.sum(axis = 1)
crimes_2018['total_crimes'] = crimes_2018.sum(axis = 1)
crimes_2019['total_crimes'] = crimes_2019.sum(axis = 1)
crimes_2020['total_crimes'] = crimes_2020.sum(axis = 1)
crimes_2020.head()

,by minors,previously committed crimes,committed by the group,intoxicating,total_crimes
region,,,,,
Белгородская область,219,7386,675,2300,10580
Брянская область,197,8171,691,3115,12174
Владимирская область,378,8927,658,3100,13063
Воронежская область,481,11825,1172,4186,17664
Ивановская область,300,6393,549,2333,9575


In [40]:
def severity_of_the_crime(crimes):
    # Обработаем наименование и оставим только часть колонок по преступлениям
    crimes.rename({
        'Unnamed: 1': 'region',
        'Unnamed: 2': 'by minors',
        'Unnamed: 3': 'previously',
        'ранее судимыми': 'previous convictions',
        'Unnamed: 5': 'group of people',
        'Unnamed: 6': 'group by prior agreement',
        'Unnamed: 7': 'organized group',
        'Unnamed: 8': 'criminal organization',
        'алкогольного': 'alcohol intoxication',
        'наркотического': 'drug intoxication',
        'токсического': 'toxic intoxication',
        'Unnamed: 12': 'crime by a family member',
        'женщин': 'against women',
        'несовершен-нолетних': 'against minor children',
        'Unnamed: 15': 'foreign citizens',
        'гражданами государств – участников СНГ': 'CIS citizens',
        'трудовых мигрантов': 'migrant workers',
        'незаконных мигрантов': 'illegal migrants',
        }, axis=1, inplace=True)
    crimes.drop([0, 1], inplace=True)
    
    # удаляем лишние столбцы
    crimes = crimes.drop(columns=['Unnamed: 0'])
    # удалим пробелы в названиях регионов
    crimes['region'] = crimes['region'].str.strip()
    # убираем строки содержащие "Российская Федерация", федеральный округ и район
    region_for_delete = crimes[crimes['region'].str.contains('федеральный округ|Федерация|район|УТ МВД по |Транспорт России')].index
    crimes.drop(region_for_delete, axis = 0, inplace=True)

    # меняем название регионов в соответствии с нашим словарем
    crimes['region'] = crimes['region'].apply(get_standard_name)
    
    crimes['previously committed crimes'] = crimes['previously'] + crimes['previous convictions']
    crimes['committed by the group'] = crimes['group of people'] + crimes['group by prior agreement'] + \
        crimes['organized group'] + crimes['criminal organization']
    crimes['intoxicating'] = crimes['alcohol intoxication'] + crimes['drug intoxication'] + \
        crimes['toxic intoxication']
    crimes['crimes family'] = crimes['crime by a family member'] + crimes['against women'] + \
        crimes['against minor children']
    crimes['crimes foreign citizens'] = crimes['foreign citizens'] + crimes['CIS citizens']
    crimes['crimes migrant'] = crimes['migrant workers'] + crimes['illegal migrants']
    # удаляем лишние столбцы
    crimes = crimes.drop(columns=['group of people', 'organized group', 'group by prior agreement','criminal organization',\
                                  'previously', 'previous convictions','crime by a family member', 'against women',  \
                                  'against minor children','alcohol intoxication', 'drug intoxication', \
                                  'toxic intoxication','migrant workers', 'illegal migrants', \
                                  'foreign citizens', 'CIS citizens'])
    crimes.set_index('region', inplace=True)
    return crimes

In [41]:
_crimes_122021 = pd.read_excel('data/crimes/4-EGS_Razdel_5_122021.xls', skiprows=10, sheet_name='Строка 1')
_crimes_122021 = severity_of_the_crime(_crimes_122021)
_crimes_122021['total_crimes_old'] =_crimes_122021['by minors'] + _crimes_122021['previously committed crimes'] + \
        _crimes_122021['committed by the group'] + _crimes_122021['intoxicating'] 
_crimes_122021['total_crimes_new'] =_crimes_122021.sum(axis = 1)
_crimes_122021.head()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,by minors,previously committed crimes,committed by the group,intoxicating,crimes family,crimes foreign citizens,crimes migrant,total_crimes_old,total_crimes_new
region,,,,,,,,,
Белгородская область,231,7511,635,2136,1865,388,88,10513,23367
Брянская область,189,9117,796,2865,2823,519,115,12967,29391
Владимирская область,334,8123,611,2694,2331,501,87,11762,26443
Воронежская область,414,13279,1194,3911,2777,622,34,18798,41029
Ивановская область,207,6414,487,2124,2387,174,1,9232,21026


Проведем предварительную подготовку датасета с учетом введения новых признаков:

In [42]:
crimes_join_2016 = pd.concat([
    crimes_2016[['total_crimes']],
    population[[2016]].rename({2016: 'population'}, axis=1)],
    axis=1)
crimes_join_2017 = pd.concat([
    crimes_2017[['total_crimes']],
    population[[2017]].rename({2017: 'population'}, axis=1)],
    axis=1)
crimes_join_2018 = pd.concat([
    crimes_2018[['total_crimes']],
    population[[2018]].rename({2018: 'population'}, axis=1)],
    axis=1)
crimes_join_2019 = pd.concat([
    crimes_2019[['total_crimes']],
    population[[2019]].rename({2019: 'population'}, axis=1)],
    axis=1)
crimes_join_2020 = pd.concat([
    crimes_2020[['total_crimes']],
    population[[2020]].rename({2020: 'population'}, axis=1)],
    axis=1)
crimes_join_2021 = pd.concat([
    _crimes_122021[['total_crimes_old']].rename({'total_crimes_old': 'total_crimes'}, axis=1),
    population[[2021]].rename({2021: 'population'}, axis=1)],
    axis=1)
crimes_join_2021.head()

,total_crimes,population
region,,
Белгородская область,10513,1541259.0
Брянская область,12967,1182682.0
Владимирская область,11762,1342099.0
Воронежская область,18798,2305608.0
Ивановская область,9232,987032.0


Введем новые признаки: 

#### 1.15.1. Уровень преступлений на 100 000 человек

Уровень криминала в регионах рассчитывается путём соотнесения общего количества учтённых преступлений с численностью населения:
<center>$$ CR = TC/P * 100 000, $$ 
где CR — уровень преступлений на 100 000 человек, 
ТС — общее количество преступлений, 
P — общая численность населения данного района;

In [43]:
crimes_join_2016['CR'] = crimes_join_2016['total_crimes']*100000 / crimes_join_2016['population']
crimes_join_2017['CR'] = crimes_join_2017['total_crimes']*100000 / crimes_join_2017['population']
crimes_join_2018['CR'] = crimes_join_2018['total_crimes']*100000 / crimes_join_2018['population']
crimes_join_2019['CR'] = crimes_join_2019['total_crimes']*100000 / crimes_join_2019['population']
crimes_join_2020['CR'] = crimes_join_2020['total_crimes']*100000 / crimes_join_2017['population']
crimes_join_2021['CR'] = crimes_join_2021['total_crimes']*100000 / crimes_join_2017['population']
crimes_join_2021.head()

,total_crimes,population,CR
region,,,
Белгородская область,10513,1541259.0,677.006694
Брянская область,12967,1182682.0,1062.407315
Владимирская область,11762,1342099.0,846.431237
Воронежская область,18798,2305608.0,804.912889
Ивановская область,9232,987032.0,902.293851


In [44]:
# объединяем датасеты
crimes_join = crimes_join_2016.join(crimes_join_2017, rsuffix='_2017', how='left')
crimes_join = crimes_join.join(crimes_join_2018, rsuffix='_2018', how='left')
crimes_join = crimes_join.join(crimes_join_2019, rsuffix='_2019', how='left')
crimes_join = crimes_join.join(crimes_join_2020, rsuffix='_2020', how='left')
crimes_join = crimes_join.join(crimes_join_2021, rsuffix='_2021', how='left')
crimes_join = crimes_join.drop(columns=['total_crimes', 'population', 'total_crimes_2017','population_2017',\
                                  'total_crimes_2018', 'population_2018','total_crimes_2019', 'population_2019',  \
                                  'total_crimes_2020','population_2020', 'total_crimes_2021', 'population_2021'])
crimes_join.rename({'CR': 'CR_2016'}, axis=1, inplace=True)
crimes_join.head()

,CR_2016,CR_2017,CR_2018,CR_2019,CR_2020,CR_2021
region,,,,,,
Белгородская область,671.230994,640.879922,649.213227,660.390405,681.321300,677.006694
Брянская область,1073.962607,975.969456,930.732249,951.768349,997.435540,1062.407315
Владимирская область,1208.802377,1147.885109,1112.282410,934.979737,940.055368,846.431237
Воронежская область,846.719295,853.983544,807.792377,782.147768,756.356063,804.912889
Ивановская область,1029.385204,953.116295,946.339906,949.829712,935.817117,902.293851


In [45]:
# усредним данные признака по годам
crimes_join_mean = crimes_join.mean(axis=1).round(1)
crimes_join_mean

region
Белгородская область           663.3
Брянская область               998.7
Владимирская область          1031.7
Воронежская область            808.7
Ивановская область             952.8
                               ...  
Приморский край               1309.3
Республика Саха (Якутия)      1237.8
Сахалинская область           1528.4
Хабаровский край              1424.5
Чукотский автономный округ    1557.6
Length: 85, dtype: float64

#### 1.15.2. Темп роста (снижения) преступности

Темп роста или уменьшения преступности (Тр), который вычисляется по следующей формуле: 
<center>$$ Tр = Y/y * 100%, $$
где Y – объем преступности за определенный временной промежуток; 
    у – объем преступности за аналогичный предшествующий временной промежуток. 

In [46]:
pace_of_crime = pd.concat([
    (pd.DataFrame(crimes_join_2017['total_crimes']* 100 / crimes_join_2016['total_crimes'])
        .rename({'total_crimes': 'pace_of_crime_1'}, axis=1)),
    (pd.DataFrame(crimes_join_2018['total_crimes']* 100 / crimes_join_2017['total_crimes'])
        .rename({'total_crimes': 'pace_of_crime_2'}, axis=1)),
    (pd.DataFrame(crimes_join_2019['total_crimes']* 100 / crimes_join_2018['total_crimes'])
        .rename({'total_crimes': 'pace_of_crime_3'}, axis=1)),
    (pd.DataFrame(crimes_join_2020['total_crimes']* 100 / crimes_join_2019['total_crimes'])
        .rename({'total_crimes': 'pace_of_crime_4'}, axis=1)),
    (pd.DataFrame(crimes_join_2021['total_crimes']* 100 / crimes_join_2020['total_crimes'])
        .rename({'total_crimes': 'pace_of_crime_4'}, axis=1))
    ], 
    axis=1)
pace_of_crime

,pace_of_crime_1,pace_of_crime_2,pace_of_crime_3,pace_of_crime_4,pace_of_crime_4
region,,,,,
Белгородская область,95.646324,101.105305,101.560326,103.532635,99.366730
Брянская область,90.489213,94.618872,101.348594,106.574455,106.513882
Владимирская область,94.446089,96.113096,83.295284,102.294440,90.040573
Воронежская область,100.941391,94.524669,96.578612,97.017631,106.419837
Ивановская область,91.991322,98.461854,99.333472,100.387922,96.417755
...,...,...,...,...,...
Приморский край,88.749017,91.969563,92.437144,97.440799,101.988763
Республика Саха (Якутия),99.337481,92.322082,104.933486,97.405759,103.430443
Сахалинская область,92.215786,97.317105,89.671867,107.378524,105.782123


In [47]:
# усредним данные признака по годам
pace_of_crime_mean = pace_of_crime.mean(axis=1).round(1)
pace_of_crime_mean

region
Белгородская область          100.2
Брянская область               99.9
Владимирская область           93.2
Воронежская область            99.1
Ивановская область             97.3
                              ...  
Приморский край                94.5
Республика Саха (Якутия)       99.5
Сахалинская область            98.5
Хабаровский край               93.2
Чукотский автономный округ    100.9
Length: 85, dtype: float64

#### 1.15.3. Темп прироста преступности

Темп прироста преступности, который определяется как процентная величина и отражает рост или снижение объема преступности в определенном временном периоде в соответствии с предшествующими периодами: 
<center> $$ Tпр = Тр -100 % $$

In [48]:
crime_growth_rate = pace_of_crime - 100
crime_growth_rate

,pace_of_crime_1,pace_of_crime_2,pace_of_crime_3,pace_of_crime_4,pace_of_crime_4
region,,,,,
Белгородская область,-4.353676,1.105305,1.560326,3.532635,-0.633270
Брянская область,-9.510787,-5.381128,1.348594,6.574455,6.513882
Владимирская область,-5.553911,-3.886904,-16.704716,2.294440,-9.959427
Воронежская область,0.941391,-5.475331,-3.421388,-2.982369,6.419837
Ивановская область,-8.008678,-1.538146,-0.666528,0.387922,-3.582245
...,...,...,...,...,...
Приморский край,-11.250983,-8.030437,-7.562856,-2.559201,1.988763
Республика Саха (Якутия),-0.662519,-7.677918,4.933486,-2.594241,3.430443
Сахалинская область,-7.784214,-2.682895,-10.328133,7.378524,5.782123


In [49]:
# усредним данные признака 
crime_growth_rate_mean = crime_growth_rate.mean(axis=1).round(1)
crime_growth_rate_mean
# удалим признак pace_of_crime_mean

region
Белгородская область          0.2
Брянская область             -0.1
Владимирская область         -6.8
Воронежская область          -0.9
Ивановская область           -2.7
                             ... 
Приморский край              -5.5
Республика Саха (Якутия)     -0.5
Сахалинская область          -1.5
Хабаровский край             -6.8
Чукотский автономный округ    0.9
Length: 85, dtype: float64

#### 1.15.4. Удельный вес тяжких преступлений

Удельный вес тяжких преступлений (L) определяется соотношением:
<center>$$ L= C/U *100 % $$
где С – количественный показатель объема тяжкой преступности; U – количественный показатель объема всей преступности.

In [50]:
# выделим строки 2 и 3 из Формы 4-ЕГС (тяжкие и особо тяжкие преступления)
particularly_serious_crimes_2021 = pd.read_excel('data/crimes/4-EGS_Razdel_5_122021.xls', skiprows=10, sheet_name='Строка 2')
particularly_serious_crimes_2020 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122020.xls', skiprows=10, sheet_name='Строка 2') 
particularly_serious_crimes_2019 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122019.xls', skiprows=10, sheet_name='Строка 2') 
particularly_serious_crimes_2018 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122018.xls', skiprows=10, sheet_name='Строка 2')
particularly_serious_crimes_2017 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122017.xls', skiprows=10, sheet_name='Строка 2')
particularly_serious_crimes_2016 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122016.xls', skiprows=10, sheet_name='Строка 2')

# обработаем все датасеты 
particularly_serious_crimes_2021 = severity_of_the_crime(particularly_serious_crimes_2021)
particularly_serious_crimes_2020 = criminal_processing(particularly_serious_crimes_2020)
particularly_serious_crimes_2019 = criminal_processing(particularly_serious_crimes_2019)
particularly_serious_crimes_2018 = criminal_processing(particularly_serious_crimes_2018)
particularly_serious_crimes_2017 = criminal_processing(particularly_serious_crimes_2017)
particularly_serious_crimes_2016 = criminal_processing(particularly_serious_crimes_2016)

particularly_serious_crimes_2021['total_crimes'] = particularly_serious_crimes_2021['by minors'] + \
    particularly_serious_crimes_2021['previously committed crimes'] + particularly_serious_crimes_2021['committed by the group'] +\
    particularly_serious_crimes_2021['intoxicating']
particularly_serious_crimes_2020['total_crimes'] = particularly_serious_crimes_2020.sum(axis = 1)
particularly_serious_crimes_2019['total_crimes'] = particularly_serious_crimes_2019.sum(axis = 1)
particularly_serious_crimes_2018['total_crimes'] = particularly_serious_crimes_2018.sum(axis = 1)
particularly_serious_crimes_2017['total_crimes'] = particularly_serious_crimes_2017.sum(axis = 1)
particularly_serious_crimes_2016['total_crimes'] = particularly_serious_crimes_2016.sum(axis = 1)

particularly_serious_crimes_2021.head()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (122053) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (122053) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,by minors,previously committed crimes,committed by the group,intoxicating,crimes family,crimes foreign citizens,crimes migrant,total_crimes
region,,,,,,,,
Белгородская область,13,225,253,91,113,137,56,582
Брянская область,31,439,377,85,45,79,6,932
Владимирская область,22,283,176,105,63,69,26,586
Воронежская область,67,434,308,196,64,100,13,1005
Ивановская область,8,229,134,87,58,43,0,458


In [51]:
# выделим строки 2 и 3 из Формы 4-ЕГС (тяжкие и особо тяжкие преступления)

serious_crimes_2021 = pd.read_excel('data/crimes/4-EGS_Razdel_5_122021.xls', skiprows=10, sheet_name='Строка 3')
serious_crimes_2020 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122020.xls', skiprows=10, sheet_name='Строка 3') 
serious_crimes_2019 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122019.xls', skiprows=10, sheet_name='Строка 3') 
serious_crimes_2018 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122018.xls', skiprows=10, sheet_name='Строка 3')
serious_crimes_2017 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122017.xls', skiprows=10, sheet_name='Строка 3')
serious_crimes_2016 = pd.read_excel('data/crimes/4-EGS_Razdel_4_122016.xls', skiprows=10, sheet_name='Строка 3')

# обработаем все датасеты 

serious_crimes_2021 = severity_of_the_crime(serious_crimes_2021)
serious_crimes_2020 = criminal_processing(serious_crimes_2020)
serious_crimes_2019 = criminal_processing(serious_crimes_2019)
serious_crimes_2018 = criminal_processing(serious_crimes_2018)
serious_crimes_2017 = criminal_processing(serious_crimes_2017)
serious_crimes_2016 = criminal_processing(serious_crimes_2016)

serious_crimes_2021['total_crimes'] = serious_crimes_2021['by minors'] + serious_crimes_2021['previously committed crimes'] + \
    serious_crimes_2021['committed by the group'] + serious_crimes_2021['intoxicating']
serious_crimes_2020['total_crimes'] = serious_crimes_2020.sum(axis = 1)
serious_crimes_2019['total_crimes'] = serious_crimes_2019.sum(axis = 1)
serious_crimes_2018['total_crimes'] = serious_crimes_2018.sum(axis = 1)
serious_crimes_2017['total_crimes'] = serious_crimes_2017.sum(axis = 1)
serious_crimes_2016['total_crimes'] = serious_crimes_2016.sum(axis = 1)

serious_crimes_2021.head()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (122053) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** file size (122053) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


,by minors,previously committed crimes,committed by the group,intoxicating,crimes family,crimes foreign citizens,crimes migrant,total_crimes
region,,,,,,,,
Белгородская область,49,1002,130,262,61,55,6,1443
Брянская область,34,1000,199,269,68,66,5,1502
Владимирская область,62,1344,131,425,88,96,15,1962
Воронежская область,79,2109,267,601,119,119,6,3056
Ивановская область,35,1040,144,304,88,29,0,1523


In [52]:
# объединим датасеты для расчета критерия
serious_crimes_join = pd.concat([
    (pd.DataFrame((particularly_serious_crimes_2016['total_crimes'] + serious_crimes_2016['total_crimes'])/crimes_join_2016['total_crimes'])
         .rename({'total_crimes': 'serious_crimes_2016'}, axis=1)),
    (pd.DataFrame((particularly_serious_crimes_2017['total_crimes'] + serious_crimes_2017['total_crimes'])/crimes_join_2017['total_crimes'])
         .rename({'total_crimes': 'serious_crimes_2017'}, axis=1)),
    (pd.DataFrame((particularly_serious_crimes_2018['total_crimes'] + serious_crimes_2018['total_crimes'])/crimes_join_2018['total_crimes'])
         .rename({'total_crimes': 'serious_crimes_2018'}, axis=1)),
    (pd.DataFrame((particularly_serious_crimes_2019['total_crimes'] + serious_crimes_2019['total_crimes'])/crimes_join_2019['total_crimes'])
         .rename({'total_crimes': 'serious_crimes_2019'}, axis=1)),
    (pd.DataFrame((particularly_serious_crimes_2020['total_crimes'] + serious_crimes_2020['total_crimes'])/crimes_join_2020['total_crimes'])
         .rename({'total_crimes': 'serious_crimes_2020'}, axis=1)),
    (pd.DataFrame((particularly_serious_crimes_2021['total_crimes'] + serious_crimes_2021['total_crimes'])/crimes_join_2021['total_crimes'])
         .rename({'total_crimes': 'serious_crimes_2021'}, axis=1))
    ], 
    axis=1)
serious_crimes_join.round(3)


,serious_crimes_2016,serious_crimes_2017,serious_crimes_2018,serious_crimes_2019,serious_crimes_2020,serious_crimes_2021
region,,,,,,
Белгородская область,0.193,0.157,0.162,0.159,0.179,0.193
Брянская область,0.195,0.193,0.186,0.176,0.171,0.188
Владимирская область,0.265,0.249,0.211,0.229,0.226,0.217
Воронежская область,0.216,0.206,0.193,0.202,0.197,0.216
Ивановская область,0.251,0.224,0.182,0.207,0.227,0.215
...,...,...,...,...,...,...
Приморский край,0.225,0.214,0.210,0.217,0.223,0.240
Республика Саха (Якутия),0.221,0.209,0.202,0.250,0.228,0.249
Сахалинская область,0.183,0.176,0.169,0.195,0.200,0.221


In [53]:
# усредним данные признака по годам
serious_crimes_join_mean = serious_crimes_join.mean(axis=1).round(3)
serious_crimes_join_mean

region
Белгородская область          0.174
Брянская область              0.185
Владимирская область          0.233
Воронежская область           0.205
Ивановская область            0.217
                              ...  
Приморский край               0.221
Республика Саха (Якутия)      0.226
Сахалинская область           0.191
Хабаровский край              0.198
Чукотский автономный округ    0.204
Length: 85, dtype: float64

#### 1.15.5. Коэффициент географии преступности

Коэффициент географии преступности (Тер), или коэффициент территориального распределения преступности, определяется следующим соотношением:
<center>$$ Тер =L/U $$
где L – показатель объема преступности на территории одного субъекта; U – показатель объема преступности на всей территории государства.

In [54]:
tep = pd.concat([
    (pd.DataFrame(crimes_join_2016['total_crimes'] / crimes_join_2016['total_crimes'].sum(axis = 0))
        .rename({'total_crimes': 'tep_2016'}, axis=1)),
    (pd.DataFrame(crimes_join_2017['total_crimes'] / crimes_join_2017['total_crimes'].sum(axis = 0))
        .rename({'total_crimes': 'tep_2017'}, axis=1)),
    (pd.DataFrame(crimes_join_2018['total_crimes'] / crimes_join_2018['total_crimes'].sum(axis = 0))
        .rename({'total_crimes': 'tep_2018'}, axis=1)),
    (pd.DataFrame(crimes_join_2019['total_crimes'] / crimes_join_2019['total_crimes'].sum(axis = 0))
        .rename({'total_crimes': 'tep_2019'}, axis=1)),
    (pd.DataFrame(crimes_join_2020['total_crimes'] / crimes_join_2020['total_crimes'].sum(axis = 0))
        .rename({'total_crimes': 'tep_2020'}, axis=1)),
    (pd.DataFrame(crimes_join_2021['total_crimes'] / crimes_join_2021['total_crimes'].sum(axis = 0))
        .rename({'total_crimes': 'tep_2021'}, axis=1))
    ], 
    axis=1)
tep.round(5)

,tep_2016,tep_2017,tep_2018,tep_2019,tep_2020,tep_2021
region,,,,,,
Белгородская область,0.00637,0.00654,0.00686,0.00726,0.00758,0.00771
Брянская область,0.00806,0.00783,0.00769,0.00811,0.00873,0.00951
Владимирская область,0.01033,0.01048,0.01046,0.00907,0.00936,0.00863
Воронежская область,0.01209,0.01311,0.01286,0.01293,0.01266,0.01379
Ивановская область,0.00649,0.00641,0.00655,0.00677,0.00686,0.00677
...,...,...,...,...,...,...
Приморский край,0.01867,0.01779,0.01698,0.01634,0.01608,0.01678
Республика Саха (Якутия),0.00757,0.00808,0.00774,0.00846,0.00832,0.00880
Сахалинская область,0.00507,0.00502,0.00507,0.00473,0.00513,0.00556


In [55]:
# усредним данные признака по годам
tep_mean = tep.mean(axis=1).round(5)
tep_mean

region
Белгородская область          0.00705
Брянская область              0.00832
Владимирская область          0.00972
Воронежская область           0.01291
Ивановская область            0.00664
                               ...   
Приморский край               0.01711
Республика Саха (Якутия)      0.00816
Сахалинская область           0.00510
Хабаровский край              0.01290
Чукотский автономный округ    0.00053
Length: 85, dtype: float64

#### 1.15.7. Объединение датасетов по преступности

In [56]:
# объединяем для создания нового датасета, признак pace_of_crime в датасет не включаем, поскольку участвует 
# в расчете темпа прироста преступностей (crime_growth_rate)

crimes = pd.concat([crimes_join_mean, crime_growth_rate_mean,  
                    serious_crimes_join_mean, tep_mean], axis=1).fillna(0)

# имена наших колонок
crimes.columns=['crime_rate', 'crime_growth_rate', 'serious_crimes', 'сrime_geography']

display(crimes)

,crime_rate,crime_growth_rate,serious_crimes,сrime_geography
region,,,,
Белгородская область,663.3,0.2,0.174,0.00705
Брянская область,998.7,-0.1,0.185,0.00832
Владимирская область,1031.7,-6.8,0.233,0.00972
Воронежская область,808.7,-0.9,0.205,0.01291
Ивановская область,952.8,-2.7,0.217,0.00664
...,...,...,...,...
Приморский край,1309.3,-5.5,0.221,0.01711
Республика Саха (Якутия),1237.8,-0.5,0.226,0.00816
Сахалинская область,1528.4,-1.5,0.191,0.00510


#### 1.16. Дети 
Данные портала https://tochno.st

In [57]:
children_2017 = pd.read_excel(
    'data/report_orphanhood_2017.xlsx',
    header=1, usecols='B:C')
children_2017.rename({'Регион': 'region'}, axis=1, inplace=True)
children_2018 = pd.read_excel(
    'data/report_orphanhood_2018.xlsx',
    header=1, usecols='B:C')
children_2018.rename({'Регион': 'region'}, axis=1, inplace=True)
children_2019 = pd.read_excel(
    'data/report_orphanhood_2019.xlsx',
    header=1, usecols='B:C')
children_2019.rename({'Регион': 'region'}, axis=1, inplace=True)
children_2020 = pd.read_excel(
    'data/report_orphanhood_2020.xlsx',
    header=1, usecols='B:C')
children_2020.rename({'Регион': 'region'}, axis=1, inplace=True)
children_2021 = pd.read_excel(
    'data/report_orphanhood_2021.xlsx',
    header=1, usecols='B:C')
children_2021.rename({'Регион': 'region'}, axis=1, inplace=True)
children_2017 = federal_subject(children_2017, standard_names)
children_2018 = federal_subject(children_2018, standard_names)
children_2019 = federal_subject(children_2019, standard_names)
children_2020 = federal_subject(children_2020, standard_names)
children_2021 = federal_subject(children_2021, standard_names)

# объединяем датасеты
children = children_2017.join(children_2018, rsuffix='_2018', how='left')
children = children.join(children_2019, rsuffix='_2019', how='left')
children = children.join(children_2020, rsuffix='_2020', how='left')
children = children.join(children_2021, rsuffix='_2021', how='left')

children.head()

,"Всего детей в регионе, человек","Всего детей в регионе, человек_2018","Всего детей в регионе, человек_2019","Всего детей в регионе, человек_2020","Всего детей в регионе, человек_2021"
region,,,,,
Алтайский край,485710,486482,483696,481809,478022
Амурская область,179041,180162,179596,178256,176234
Архангельская область без Ненецкого АО,212129,211293,209204,206120,201724
Астраханская область,224268,226337,227234,226853,226112
Белгородская область,284099,287196,288799,290617,290070


In [58]:
# усредним данные признака в динамике по годам
children_mean = children.mean(axis=1).round()
children_mean

region
Алтайский край                            483144.0
Амурская область                          178658.0
Архангельская область без Ненецкого АО    208094.0
Астраханская область                      226161.0
Белгородская область                      288156.0
                                            ...   
Чеченская Республика                      543188.0
Чувашская Республика                      256203.0
Чукотский автономный округ                 12228.0
Ямало-Ненецкий автономный округ           129802.0
Ярославская область                       239700.0
Length: 85, dtype: float64

#### 1.17. Пенсионеры 
Данные портала https://tochno.st

In [59]:
retired_2019 = pd.read_excel(
    'data/report_ageing_2019.xlsx',
    header=1, usecols='B:I')
retired_2019.rename({'Регион': 'region'}, axis=1, inplace=True)
retired_2019 = federal_subject(retired_2019, standard_names)

retired_2020 = pd.read_excel(
    'data/report_ageing_2020.xlsx',
    header=1, usecols='B:I')
retired_2020.rename({'Регион': 'region'}, axis=1, inplace=True)
retired_2020 = federal_subject(retired_2020, standard_names)

In [60]:
# объединим датасеты для расчета критерия
retired = pd.concat([
    (pd.DataFrame((retired_2019['Ожидаемая продолжительность жизни в 55 лет без инвалидности, год'] + \
                   retired_2020['Ожидаемая продолжительность жизни в 55 лет без инвалидности, год'])/2)\
         .rename({'Ожидаемая продолжительность жизни в 55 лет без инвалидности, год': 'life_expectancy'}, axis=1)),
    (pd.DataFrame((retired_2019['Уровень занятости пожилых, %'] + retired_2020['Уровень занятости пожилых, %'])/2)
         .rename({'Уровень занятости пожилых, %': 'employment_rate'}, axis=1)),
    (pd.DataFrame((retired_2019['Занятые в неформальном секторе экономики, %'] + retired_2020['Занятые в неформальном секторе экономики, %'])/2)
         .rename({'Занятые в неформальном секторе экономики, %': 'busy_retired'}, axis=1)),
    (pd.DataFrame((retired_2019['Пожилые люди с доходами ниже прожиточного минимума, %'] + \
                   retired_2020['Пожилые люди с доходами ниже прожиточного минимума, %'])/2)\
         .rename({'Пожилые люди с доходами ниже прожиточного минимума, %': 'below_the_subsistence_level'}, axis=1)),
    (pd.DataFrame((retired_2019['Бедные по самооценке пожилые люди, %'] + retired_2020['Бедные по самооценке пожилые люди, %'])/2)
         .rename({'Бедные по самооценке пожилые люди, %': 'poor_elderly'}, axis=1)),
    (pd.DataFrame((retired_2019['Охват пожилых соц. обслуживанием, чел./100000'] + \
                   retired_2020['Охват пожилых соц. обслуживанием, чел./100000'])/2)
         .rename({'Охват пожилых соц. обслуживанием, чел./100000': 'coverage_of_the_elderly'}, axis=1)),
    (pd.DataFrame((retired_2019['Посещаемость пожилыми досуговых мероприятий, %'] + \
                   retired_2020['Посещаемость пожилыми досуговых мероприятий, %'])/2)
         .rename({'Посещаемость пожилыми досуговых мероприятий, %': 'leisure_attendance'}, axis=1)),             
    ], 
    axis=1)
retired.round(3)

,life_expectancy,employment_rate,busy_retired,below_the_subsistence_level,poor_elderly,coverage_of_the_elderly,leisure_attendance
region,,,,,,,
Алтайский край,17.55,16.20,29.00,3.50,26.35,3746.0,28.90
Амурская область,15.05,19.35,19.00,3.05,31.05,4883.5,44.50
Архангельская область без Ненецкого АО,17.65,16.25,17.65,2.40,16.45,4704.5,46.75
Астраханская область,20.45,18.40,27.05,2.90,9.20,5793.0,34.85
Белгородская область,14.60,23.80,39.60,2.00,0.65,7537.0,35.95
...,...,...,...,...,...,...,...
Чеченская Республика,15.20,31.40,64.05,29.00,19.35,11993.0,37.35
Чувашская Республика,18.60,18.15,27.65,2.45,8.80,3172.0,39.70
Чукотский автономный округ,17.25,56.40,3.20,0.50,10.30,6219.5,38.50


#### 1.18. Формирование сводной таблицы 

In [61]:
# объединим все данные в один датасет
data = pd.concat([per_capita_mean, real_incomes_mean, formal_wage_mean, real_pay_mean, 
                  child_mortality_mean, disabled_mean, alco_mean, drug_mean,
                  gross_regional_product_mean, housing, general_morbidity_mean,
                  congenital_malformation_mean, psychosis_mean, teenage_pregnancy_mean,
                  newborn_mean, population_mean, poverty_percent_mean, poverty_socdem,
                  reg_prod_total_mean, retail_mean, welfare_mean, crimes, children_mean, retired],
                  axis=1).fillna(0)

# имена наших колонок
data.columns=['per_capita','real_incomes', 'formal_wage', 'real_pay', 
              'child_mortality', 'disabled', 'alco', 'drug', 'gross_regional_product',
              'living_area', 'bad_conditions', 'no_funds', 'general_morbidity',
              'congenital_malformation', 'psychosis', 'teenage_pregnancy', 'newborn', 'population', 
              'poverty_percent', 'the_poor: children', 'the_poor: retired', 'the_poor: employable', 'reg_prod_total',
              'retail', 'welfare', 'crime_rate', 'crime_growth_rate', 'serious_crimes', 'сrime_geography', 'children',  
              'life_expectancy', 'employment_rate', 'busy_retired', 'below_the_subsistence_level', 
              'poor_elderly', 'coverage_of_the_elderly', 'leisure_attendance']

display(data)

,per_capita,real_incomes,formal_wage,real_pay,child_mortality,disabled,alco,drug,gross_regional_product,living_area,...,serious_crimes,сrime_geography,children,life_expectancy,employment_rate,busy_retired,below_the_subsistence_level,poor_elderly,coverage_of_the_elderly,leisure_attendance
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,22480.8,98.1,24598.0,90.000000,323.7,62401.5,87.000000,17.400000,212259.9,18.0,...,0.170,0.02573,483144.0,17.55,16.20,29.00,3.50,26.35,3746.0,28.90
Амурская область,30861.5,98.9,41014.3,88.000000,174.8,25527.0,102.350006,21.549999,369814.0,17.3,...,0.225,0.01007,178658.0,15.05,19.35,19.00,3.05,31.05,4883.5,44.50
Архангельская область без Ненецкого АО,31840.0,97.9,43617.2,92.400002,143.4,27432.7,60.000000,7.300000,381017.2,16.9,...,0.220,0.01106,208094.0,17.65,16.25,17.65,2.40,16.45,4704.5,46.75
Астраханская область,23899.5,96.6,31866.5,89.900002,145.7,19912.0,24.200001,0.450000,374377.4,19.2,...,0.182,0.00827,226161.0,20.45,18.40,27.05,2.90,9.20,5793.0,34.85
Белгородская область,30692.5,99.4,30920.3,93.199997,139.2,62646.7,43.250000,2.750000,479672.1,17.8,...,0.174,0.00705,288156.0,14.60,23.80,39.60,2.00,0.65,7537.0,35.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,23330.5,99.1,25364.0,88.199997,369.0,135625.3,3.550000,0.700000,124347.6,19.2,...,0.250,0.00101,543188.0,15.20,31.40,64.05,29.00,19.35,11993.0,37.35
Чувашская Республика,18919.7,98.9,26226.3,89.599998,135.1,29728.8,77.699997,1.450000,219475.7,14.2,...,0.246,0.00814,256203.0,18.60,18.15,27.65,2.45,8.80,3172.0,39.70
Чукотский автономный округ,75961.8,101.5,97464.2,96.300003,19.5,1062.8,231.399994,1.000000,1392794.6,14.1,...,0.204,0.00053,12228.0,17.25,56.40,3.20,0.50,10.30,6219.5,38.50


Собираем датасет с усредненными данными по годам.  

Описание финального датасета:
| Признак | Описание |
| - | - |
| per_capita | среднемесячный денежный доход на человека, в рублях|
| real_incomes | средние реальные денежные доходы, в процентах к предыдущему году|
| formal_wage | среднемесячная номинальная начисленная заработная плата на человека, в рублях|
| real_pay | средняя реальная начисленная заработная плата, в процентах к предыдущему году|
| child_mortality | средняя детская смертность, в процентах от количества новорожденных|
| disabled | средняя инвалидность среди трудоспособного возраста, в процентах к населению соответствующего возраста (оценочно)|
| alco | среднее количество впервые зарегистрированных алкогольных психозов на 100 тыс. населения|
| drug | среднее количество впервые зарегистрированных наркотических психозов на 100 тыс. населения|
| gross_regional_product | средний валовый региональный продукт (объем произведенных товаров и услуг) на душу населения, в рублях|
| living_area | средняя жилая площадь на человека|
| bad_conditions | процент респондентов, указавших на плохое состояние жилья|
| no_funds | процент респондентов, рассчитывающих на получение жилья от государства|
| general_morbidity | общая заболеваемость на 100 тыс. населения|
| congenital_malformation | средние значения врожденных пороков развития на 100 тыс. населения|
| psychosis | средние значения психических расстройств на 100 тыс. населения|
| teenage_pregnancy | усредненное значение подростковой беременности на 100 тыс. населения|
| newborn | средняя рождаемость на 100 тыс. населения|
| population | среднее население, количество человек|
| poverty_percent | средний процент населения за чертой бедности|
| the_poor: children | средний процент детей среди малообеспеченного населения|
| the_poor: retired | средний процент пенсионеров среди малообеспеченного населения|
| the_poor: employable | средний процент людей трудоспособного возраста среди малообеспеченного населения|
| reg_prod_total | усредненное значение региональных производств по всем видам деятельности на душу населения, в рублях|
| retail | усредненное значение розничной торговли на душу населения, в рублях|
| welfare | средние расходы регионального бюджета на социальную политику, в рублях на душу населения|
| crime_rate | средний уровень преступлений по годам|
| pace_of_crime | средний темп роста или уменьшения преступлений|
| crime_growth_rate | средний темп прироста преступностей|
| serious_crimes | средний удельный вес тяжких преступлений|
| сrime_geography | средние значения коэффициентов территориального распределения преступности|
| children | среднее количество детей|
| life_expectancy | ожидаемая продолжительность жизни в 55 лет без инвалидности, год|
| employment_rate | уровень занятости пожилых, %|
| busy_retired | занятые в неформальном секторе экономики, %|
| below_the_subsistence_level | пожилые люди с доходами ниже прожиточного минимума, %|
| poor_elderly | бедные по самооценке пожилые люди, %|
| coverage_of_the_elderly | охват пожилых социальным обслуживанием, чел./100000|
| leisure_attendance | посещаемость пожилыми досуговых мероприятий|

In [62]:
data.to_csv('outliers/data_regions.csv')
data

,per_capita,real_incomes,formal_wage,real_pay,child_mortality,disabled,alco,drug,gross_regional_product,living_area,...,serious_crimes,сrime_geography,children,life_expectancy,employment_rate,busy_retired,below_the_subsistence_level,poor_elderly,coverage_of_the_elderly,leisure_attendance
region,,,,,,,,,,,,,,,,,,,,,
Алтайский край,22480.8,98.1,24598.0,90.000000,323.7,62401.5,87.000000,17.400000,212259.9,18.0,...,0.170,0.02573,483144.0,17.55,16.20,29.00,3.50,26.35,3746.0,28.90
Амурская область,30861.5,98.9,41014.3,88.000000,174.8,25527.0,102.350006,21.549999,369814.0,17.3,...,0.225,0.01007,178658.0,15.05,19.35,19.00,3.05,31.05,4883.5,44.50
Архангельская область без Ненецкого АО,31840.0,97.9,43617.2,92.400002,143.4,27432.7,60.000000,7.300000,381017.2,16.9,...,0.220,0.01106,208094.0,17.65,16.25,17.65,2.40,16.45,4704.5,46.75
Астраханская область,23899.5,96.6,31866.5,89.900002,145.7,19912.0,24.200001,0.450000,374377.4,19.2,...,0.182,0.00827,226161.0,20.45,18.40,27.05,2.90,9.20,5793.0,34.85
Белгородская область,30692.5,99.4,30920.3,93.199997,139.2,62646.7,43.250000,2.750000,479672.1,17.8,...,0.174,0.00705,288156.0,14.60,23.80,39.60,2.00,0.65,7537.0,35.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Чеченская Республика,23330.5,99.1,25364.0,88.199997,369.0,135625.3,3.550000,0.700000,124347.6,19.2,...,0.250,0.00101,543188.0,15.20,31.40,64.05,29.00,19.35,11993.0,37.35
Чувашская Республика,18919.7,98.9,26226.3,89.599998,135.1,29728.8,77.699997,1.450000,219475.7,14.2,...,0.246,0.00814,256203.0,18.60,18.15,27.65,2.45,8.80,3172.0,39.70
Чукотский автономный округ,75961.8,101.5,97464.2,96.300003,19.5,1062.8,231.399994,1.000000,1392794.6,14.1,...,0.204,0.00053,12228.0,17.25,56.40,3.20,0.50,10.30,6219.5,38.50


In [63]:
# сохраним модель в виде pickle-файла
with open('data/model_1.pkl', 'wb') as pkl_file:
    pickle.dump(data, pkl_file)

## Выводы по части 1:
Проведено считывание экономических и демографических данных из предоставленных файлов различного формата и из дополнительных источников. Результаты сохранены в датасеты со стандартизованными названиями федеральных субъектов РФ (85 субъектов). На основе полученных датасетов сформирована финальная таблица по усредненным значениям 37 показателей по разным годам, которая будет использоваться при кластеризации.